# Data Manipulation Language (DML) operations in SQL. 
DML operations are used to manipulate data stored in the database tables. They allow you to add, modify, and remove records, thereby enabling you to interact with and manage the data within your database.

In [10]:
## Setup Database Connection
import sqlite3
import pandas as pd

conn = sqlite3.connect('fake_data.db')
cursor = conn.cursor()

# Execute the SQL command to list all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

# Fetch all the rows (table names)
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

names
addresses
texts
dates_and_times
internet_data


## Inserting a New Record
You want to insert a new record into a table. For example, you want to insert a new record into the names table. 

In [13]:
cursor.execute('''
               INSERT INTO names(full_name, first_name, last_name, prefix) 
               VALUES ('John Doe', 'John', 'Doe', 'Mr.')
               ''')
conn.commit()

## Inserting Default Values
A table can be defined to take default values for specific columns. You want to insert a row of default values without having to specify those values.

In [16]:
cursor.execute('''
                CREATE TABLE IF NOT EXISTS test_table 
               (id integer default 0,
                foo varchar(10))
                ''')
conn.commit()

## Overriding a Default Value with NULL
You are inserting into a column having a default value, and you want to override that default value by setting the column to NULL.

In [18]:
cursor.execute('''
               INSERT INTO test_table(id, foo) 
               VALUES (null, 'bar')
               ''')
conn.commit()

## Copying Rows from One Table into Another
You want to copy rows from one table to another by using a query. The query may be complex or simple, but ultimately you want the result to be inserted into another table.

In [20]:
cursor.execute('''INSERT INTO test_table(id, foo)
               SELECT first_name, last_name FROM names
               WHERE prefix = 'Mr.'
                ''')
conn.commit()

## Copying a Table Definition
You want to create a new table having the same set of columns as an existing table. 

In [23]:
cursor.execute('''CREATE TABLE IF NOT EXISTS test_table2
               AS
               SELECT * FROM names
               WHERE 1 = 0
                ''')
conn.commit()

## Blocking Inserts to Certain Columns
You want to prevent users, or an errant software application, from inserting values into certain table columns.
Create a view on the table exposing only those columns you want to expose. Then force all inserts to go through that view.

In [24]:
cursor.execute('''CREATE VIEW test_view 
               AS
                SELECT first_name, prefix FROM names
                 ''')
conn.commit()

## Modifying Records in a Table
You want to modify values for some or all rows in a table.


In [ ]:
# Let say you want to increase the salary for every senior position by 10 %
cursor.execute('''UPDATE names
               SET salary = salary * 1.1
               WHERE position = 'Senior'
                ''')
conn.commit()

## Updating When Corresponding Rows Exist
You want to update rows in one table when corresponding rows exist in another.

In [ ]:
# Let say you want to increase the salary for every senior position by 10 % from the health department
cursor.execute('''UPDATE names
               SET salary = salary * 1.1
               WHERE position = (SELECT position FROM healthcare WHERE department = 'Health')
                ''')
conn.commit()

## Updating with Values from Another Table
You want to update rows in one table using values from another.

In [ ]:
cursor.execute('''UPDATE names
               SET salary = AD.salary * 1.1
               FROM addresses AS AD
               WHERE AD.position = names.position
                ''')
conn.commit()